# Chapter 22: Reader

In [1]:
boop = (*2)
doop = (+10)
bip = boop . doop
bip 3

26

In [2]:
bloop = fmap boop doop
-- same as bip

bloop 3

26

In [3]:
bbop = (+) <$> boop <*> doop
bbop 3

19

In [4]:
import Control.Applicative (liftA2)

duwop = liftA2 (+) boop doop
duwop 3

19

In [5]:
wutwop x = boop x + doop x
wutwop 3

19

In [6]:
boopDoop = do
    a <- boop
    b <- doop
    return (a + b)
    
boopDoop 3

19

---

In [7]:
import Data.Char (toUpper)

cap :: [Char] -> [Char]
cap xs = map toUpper xs

rev :: [Char] -> [Char]
rev xs = reverse xs

Line 3: Use String
Found:
[Char] -> [Char]
Why not:
String -> StringLine 4: Eta reduce
Found:
cap xs = map toUpper xs
Why not:
cap = map toUpperLine 6: Use String
Found:
[Char] -> [Char]
Why not:
String -> StringLine 7: Eta reduce
Found:
rev xs = reverse xs
Why not:
rev = reverse

In [8]:
composed :: [Char] -> [Char]
composed = rev . cap

fmapped :: [Char] -> [Char]
fmapped = fmap rev cap

composed "Julie"
fmapped "Chris"

Line 1: Use String
Found:
[Char] -> [Char]
Why not:
String -> StringLine 4: Use String
Found:
[Char] -> [Char]
Why not:
String -> String

"EILUJ"

"SIRHC"

In [9]:
tupled :: [Char] -> ([Char], [Char])
tupled = liftA2 (,) rev cap

tupled "Julie"

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

("eiluJ","JULIE")

In [10]:
tupled' :: [Char] -> ([Char], [Char])
tupled' = do
    a <- rev
    b <- cap
    return (a, b)
    
tupled' "Julie"

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

("eiluJ","JULIE")

In [11]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

In [12]:
:t (>>=) :: (c -> a) -> (a -> c -> b) -> (c -> b) 

(>>=) :: (c -> a) -> (a -> c -> b) -> (c -> b) :: forall c a b. (c -> a) -> (a -> c -> b) -> c -> b

In [13]:
:t rev >>= (,)

rev >>= (,) :: [Char] -> ([Char], [Char])

In [14]:
:t cap >>= (,)

cap >>= (,) :: [Char] -> ([Char], [Char])

In [15]:
:t (,)

(,) :: forall a b. a -> b -> (a, b)

In [16]:
:t return

return :: forall (m :: * -> *) a. Monad m => a -> m a

In [17]:
:t return :: a -> c -> a

return :: a -> c -> a :: forall a c. a -> c -> a

In [18]:
tupled'' :: [Char] -> ([Char], [Char])
tupled'' = rev >>= (\a -> cap >>= (\b -> return (a, b)))

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

In [19]:
tupled'' "Julie"

("eiluJ","JULIE")

In [20]:
tupled''' :: [Char] -> ([Char], [Char])
tupled''' = rev >>= (\a b -> (a, cap b))

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

In [21]:
tupled''' "Julie"

("eiluJ","JULIE")

---

In [22]:
fmap (+1) (*2) 3

7

In [23]:
fmap (+1) (*2) $ 3

Line 1: Redundant $
Found:
fmap (+ 1) (* 2) $ 3
Why not:
fmap (+ 1) (* 2) 3

7

In [24]:
(fmap (+1) (*2)) 3

Line 1: Redundant bracket
Found:
(fmap (+ 1) (* 2)) 3
Why not:
fmap (+ 1) (* 2) 3

7

In [25]:
(+1) . (*2) $ 3

7

In [26]:
(+2) . (*1) $ 2

4

In [27]:
fmap (+2) (*1) $ 2

Line 1: Redundant $
Found:
fmap (+ 2) (* 1) $ 2
Why not:
fmap (+ 2) (* 1) 2

4

In [28]:
(+2) `fmap` (*1) $ 2

4

---

In [62]:
newtype Reader r a = Reader { runReader :: r -> a }

instance Functor (Reader r) where
    fmap f (Reader ra) = Reader $ fmap f ra

In [30]:
ask :: Reader a a
ask = Reader id

In [31]:
:t runReader

runReader :: forall r a. Reader r a -> r -> a

---

In [32]:
newtype HumanName = HumanName String deriving (Eq, Show)

newtype DogName = DogName String deriving (Eq, Show)

newtype Address = Address String deriving (Eq, Show)

In [33]:
data Person = Person
    { humanName :: HumanName
    , dogName :: DogName
    , address :: Address
    } deriving (Eq, Show)

In [34]:
data Dog = Dog
    { dogsName :: DogName
    , dogsAddress :: Address
    } deriving (Eq, Show)

In [35]:
pers :: Person
pers = Person 
    (HumanName "Big Bird") 
    (DogName "Barkley")
    (Address "Sesame Street")

In [36]:
chris :: Person
chris = Person
    (HumanName "Chris Allen")
    (DogName "Papu")
    (Address "Austin")

In [37]:
getDog :: Person -> Dog
getDog p = Dog (dogName p) (address p)

getDog pers
getDog chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

In [38]:
getDogR :: Person -> Dog
getDogR = Dog <$> dogName <*> address

getDogR pers
getDogR chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

In [39]:
getDogR' :: Person -> Dog
getDogR' = liftA2 Dog dogName address

getDogR' pers
getDogR' chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

---

In [40]:
myLiftA2 :: Applicative f => (a -> b -> c) -> f a -> f b -> f c
myLiftA2 f ma mb = f <$> ma <*> mb

In [50]:
asks :: (r -> a) -> Reader r a
asks = Reader

In [63]:
import Data.Function (on)

instance Applicative (Reader r) where
    pure = Reader . pure
    --(Reader rf) <*> (Reader ra) = Reader $ \r -> rf r (ra r)
    (Reader rf) <*> (Reader ra) = Reader $ rf <*> ra
    
runReader ((+) <$> Reader (*2) <*> Reader (*3)) 3

15

---

In [67]:
foo :: (Functor f, Num a) => f a -> f a
foo r = fmap (+1) r

foo [1, 2, 3]

Line 2: Eta reduce
Found:
foo r = fmap (+ 1) r
Why not:
foo = fmap (+ 1)

[2,3,4]

In [72]:
bar :: Foldable f => t -> f a -> (t, Int)
bar r t = (r, length t)

bar [2, 3, 4] [1, 2, 3]

([2,3,4],3)

In [73]:
froot :: Num a => [a] -> ([a], Int)
froot r = (map (+1) r, length r)

froot [1, 2, 3]

([2,3,4],3)

In [74]:
barOne :: Foldable t => t a -> (t a, Int)
barOne r = (r, length r)

barOne [1, 2, 3]

([1,2,3],3)

In [76]:
barPlus :: (Functor t, Num a, Foldable t) => t a -> (t a, Int)
barPlus r = (foo r, length r)

barPlus [1, 2, 3]

([2,3,4],3)

In [80]:
frooty :: Num a => [a] -> ([a], Int)
frooty r = bar (foo r) r

frooty [1, 2, 3]

([2,3,4],3)

In [81]:
frooty' :: Num a => [a] -> ([a], Int)
frooty' = \r -> bar (foo r) r

frooty' [1, 2, 3]

Line 2: Redundant lambda
Found:
frooty' = \ r -> bar (foo r) r
Why not:
frooty' r = bar (foo r) r

([2,3,4],3)

In [82]:
fooBind :: (r -> a) -> (a -> r -> c) -> (r -> c)
fooBind m k = \r -> k (m r) r

Line 2: Redundant lambda
Found:
fooBind m k = \ r -> k (m r) r
Why not:
fooBind m k r = k (m r) r

---

In [83]:
getDogRM :: Person -> Dog
getDogRM = do
    name <- dogName
    addy <- address
    return $ Dog name addy
    
getDogRM pers
getDogRM chris

Line 2: Use <$>
Found:
do name <- dogName
   addy <- address
   return $ Dog name addy
Why not:
do name <- dogName
   Dog name <$> address

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

---

In [84]:
:info Monad

class Applicative m => Monad (m :: * -> *) where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  fail :: String -> m a
  {-# MINIMAL (>>=) #-}
  	-- Defined in ‘GHC.Base’
instance Monad (Either e) -- Defined in ‘Data.Either’
instance Monad IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Monad [] -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Monad IO -- Defined in ‘GHC.Base’
instance Monad ((->) r) -- Defined in ‘GHC.Base’
instance Monoid a => Monad ((,) a) -- Defined in ‘GHC.Base’

In [90]:
instance Monad (Reader r) where
    (Reader ra) >>= arb = Reader $ \r -> runReader (arb (ra r)) r

In [96]:
getDogRM' :: Reader Person Dog
getDogRM' = do
    name <- Reader dogName
    addy <- Reader address
    return $ Dog name addy
    
runReader getDogRM' pers
runReader getDogRM' chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

---

## A warm-up stretch

## Rewriting Shawty